# Axis transposition benchmarking
This notebook compares performance of different implementations of transposing axes.

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
import numpy as np
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<GeForce RTX 2070 on Platform: NVIDIA CUDA (1 refs)>

In [2]:
# test data
import numpy as np

test_image = np.random.random([100, 512, 1024])

In [3]:
# transpose with numpy
result_image = None

for i in range(0, 10):
    start_time = time.time()
    result_image = np.transpose(test_image, (2, 1, 0))
    print("numpy transpose duration: " + str(time.time() - start_time))
    

numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0


In [4]:
# transpose with pyclesperanto
result_image = None

test_image_gpu = cle.push_zyx(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cle.transpose_xz(test_image_gpu, result_image)
    print("pyclesperanto transpose duration: " + str(time.time() - start_time))

pyclesperanto transpose duration: 0.5206706523895264
pyclesperanto transpose duration: 0.01561737060546875
pyclesperanto transpose duration: 0.02116107940673828
pyclesperanto transpose duration: 0.015658855438232422
pyclesperanto transpose duration: 0.031207799911499023
pyclesperanto transpose duration: 0.015656471252441406
pyclesperanto transpose duration: 0.015619516372680664
pyclesperanto transpose duration: 0.022141695022583008
pyclesperanto transpose duration: 0.031244993209838867
pyclesperanto transpose duration: 0.015657901763916016
